# Aneurysm Simulation Velocity Magnitude Analyzer

This notebook loads the pre-processed velocity magnitude data from VTK files and corresponding simulation parameters from JSON files. It provides tools for analyzing the flow patterns in aneurysm simulations based on the velocity magnitude field.

## Import Required Libraries

In [ ]:
# Import Required Libraries
import os
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import re
from IPython.display import display, clear_output
import time

# For 3D visualization 
import plotly.express as px
import plotly.graph_objects as go

## Locate and Load Simulation Parameters

First, we need to find the simulation parameters in the JSON files stored in the 'parameters' directory.

In [ ]:
# Define the base directory where the simulation results are located
base_dir = Path('/home/abdua786/code/uni/3/dissertation/dissertation')
simulation_folder = base_dir / 'aneurysm_simulation_results'

# Check if the simulation folder exists
if not simulation_folder.exists():
    print(f"Error: Simulation folder not found at {simulation_folder}")
else:
    print(f"Found simulation folder at: {simulation_folder}")
    
    # Define paths to the parameters folder and processed data folder
    params_folder = simulation_folder / 'parameters'
    processed_data_folder = simulation_folder / 'processed_data' / 'raw_fields'
    
    # Check if each folder exists
    folders_status = {
        "Parameter files": params_folder.exists(),
        "Processed data": processed_data_folder.exists()
    }
    
    for folder_type, exists in folders_status.items():
        status = "✓ Found" if exists else "✗ Not found"
        print(f"{status}: {folder_type}")
    
    # Get all parameter files
    if params_folder.exists():
        param_files = list(params_folder.glob('*.json'))
        print(f"\nFound {len(param_files)} parameter files")
        
        # Display some example filenames
        if param_files:
            print("Example parameter files:")
            for file in param_files[:3]:  # Show up to 3 files
                print(f"- {file.name}")

    # Check for processed data files
    if processed_data_folder.exists():
        npz_files = list(processed_data_folder.glob('*.npz'))
        print(f"\nFound {len(npz_files)} processed data files")
        
        # Display some example filenames
        if npz_files:
            print("Example data files:")
            for file in npz_files[:5]:  # Show up to 5 files
                print(f"- {file.name}")

## Extract Key Simulation Parameters

Now, let's load and parse the parameter files to extract the key simulation parameters such as frames per second (fps), time step (dt), and grid spacing (dx).

In [ ]:
def load_parameters(param_files):
    """Load and parse parameter files"""
    parameters = {}
    
    if not param_files:
        print("No parameter files to load")
        return parameters
    
    for file_path in param_files:
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
                # Use the filename as the key
                parameters[file_path.stem] = data
        except Exception as e:
            print(f"Error loading {file_path.name}: {e}")
    
    return parameters

# Load the parameters if parameter files were found
if 'param_files' in locals() and param_files:
    sim_parameters = load_parameters(param_files)
    
    # Display the parameters in a structured way
    if sim_parameters:
        print(f"Loaded {len(sim_parameters)} parameter files successfully")
        
        # Display parameters from the first file as an example
        first_param_file = next(iter(sim_parameters))
        params = sim_parameters[first_param_file]
        
        print(f"\nParameters from {first_param_file}:")
        # Group parameters by category if they follow a standard format
        # Otherwise, display the top-level parameters
        for key, value in params.items():
            # If the value is a dictionary, show it in a compact form
            if isinstance(value, dict):
                print(f"- {key}: {len(value)} parameters")
            elif isinstance(value, list) and len(value) > 5:
                print(f"- {key}: list with {len(value)} items")
            else:
                print(f"- {key}: {value}")
else:
    print("No parameter files available")
    sim_parameters = {}

In [ ]:
def extract_key_parameters(sim_parameters):
    """Extract the key simulation parameters needed for analysis"""
    # Initialize dictionary for key parameters
    key_params = {}
    
    # Loop through all parameter files
    for param_name, params in sim_parameters.items():
        key_params[param_name] = {}
        
        # Extract time step (dt) if available
        if 'dt' in params:
            key_params[param_name]['dt'] = params['dt']
        elif 'time_step' in params:
            key_params[param_name]['dt'] = params['time_step']
        
        # Extract grid spacing (dx) if available
        if 'dx' in params:
            key_params[param_name]['dx'] = params['dx']
        elif 'lattice_spacing' in params:
            key_params[param_name]['dx'] = params['lattice_spacing']
        
        # Extract viscosity
        if 'viscosity' in params:
            key_params[param_name]['viscosity'] = params['viscosity']
        
        # Extract domain size
        if 'domain_size' in params:
            key_params[param_name]['domain_size'] = params['domain_size']
        
        # Extract total number of time steps
        if 'total_timesteps' in params:
            key_params[param_name]['total_timesteps'] = params['total_timesteps']
        
        # Extract frames per second if available or calculate it
        if 'fps' in params:
            key_params[param_name]['fps'] = params['fps']
        elif 'output_frequency' in params and 'dt' in key_params[param_name]:
            # Calculate fps = 1 / (dt * output_frequency)
            dt = key_params[param_name]['dt']
            output_freq = params['output_frequency']
            key_params[param_name]['fps'] = 1.0 / (dt * output_freq)
        
        # Calculate Reynolds number if possible
        if all(k in key_params[param_name] for k in ['viscosity', 'dx']) and 'characteristic_velocity' in params:
            # Re = (U * L) / nu where U is characteristic velocity, L is characteristic length
            u = params['characteristic_velocity']
            l = key_params[param_name]['dx'] * params.get('characteristic_length_cells', 1)
            nu = key_params[param_name]['viscosity']
            key_params[param_name]['reynolds_number'] = (u * l) / nu
    
    return key_params

# Extract key parameters
if 'sim_parameters' in locals() and sim_parameters:
    key_params = extract_key_parameters(sim_parameters)
    
    # Display the extracted parameters
    if key_params:
        print("\nExtracted Key Parameters:")
        for param_name, params in key_params.items():
            print(f"\nParameters from {param_name}:")
            for key, value in params.items():
                print(f"- {key}: {value}")
    else:
        print("No key parameters could be extracted")
else:
    print("No simulation parameters available")
    key_params = {}

## Load Velocity Magnitude Data

Now we'll load the velocity magnitude data from the pre-processed NumPy arrays.

In [ ]:
def load_processed_data(processed_data_folder, field_name='u_magnitude'):
    """Load processed data for a specific field"""
    # Check if the folder exists
    if not processed_data_folder.exists():
        print(f"Error: Processed data folder not found at {processed_data_folder}")
        return None, None
    
    # Check if the frame mapping file exists
    frame_mapping_file = processed_data_folder / 'frame_mapping.npz'
    if not frame_mapping_file.exists():
        print(f"Error: Frame mapping file not found at {frame_mapping_file}")
        return None, None
    
    # Check if the field data file exists
    field_data_file = processed_data_folder / f'{field_name}.npz'
    if not field_data_file.exists():
        print(f"Error: Field data file not found at {field_data_file}")
        return None, None
    
    try:
        # Load frame mapping
        frame_mapping = np.load(frame_mapping_file)
        frame_numbers = frame_mapping['frame_numbers']
        frame_indices = frame_mapping['frame_indices']
        
        # Load field data
        field_data = np.load(field_data_file)['data']
        
        print(f"Successfully loaded {field_name} data with shape {field_data.shape}")
        print(f"Frame mapping contains {len(frame_numbers)} frames")
        
        # Create a more convenient mapping
        frame_to_index = {frame: idx for frame, idx in zip(frame_numbers, frame_indices)}
        
        return field_data, frame_to_index
    
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None

# Load velocity magnitude data
if 'processed_data_folder' in locals() and processed_data_folder.exists():
    velocity_data, frame_to_index = load_processed_data(processed_data_folder, 'u_magnitude')
    
    if velocity_data is not None:
        # Display some basic info about the velocity data
        print(f"\nVelocity magnitude data:")
        print(f"- Shape: {velocity_data.shape}")
        print(f"- Data type: {velocity_data.dtype}")
        print(f"- Min value: {velocity_data.min()}")
        print(f"- Max value: {velocity_data.max()}")
        print(f"- Mean value: {velocity_data.mean()}")
        
        # Show frame numbers (first few and last few)
        frame_numbers = sorted(frame_to_index.keys())
        if len(frame_numbers) > 6:
            print(f"\nFrame numbers: {frame_numbers[:3]} ... {frame_numbers[-3:]}")
        else:
            print(f"\nFrame numbers: {frame_numbers}")
else:
    print("Processed data folder not available")

In [ ]:
def reshape_field_data(field_data, dimensions_file=None):
    """Reshape 1D field data into 2D arrays based on simulation dimensions"""
    # If dimensions file is provided, load it
    if dimensions_file is not None and dimensions_file.exists():
        try:
            dimensions = np.load(dimensions_file)['data']
            # Get real dimensions (non-zero)
            real_dims = [d for d in dimensions if d > 1]
            # The dimensions are cell counts + 1 in VTK
            ny, nx = real_dims[1]-1, real_dims[0]-1
        except Exception as e:
            print(f"Error loading dimensions: {e}")
            return None
    else:
        # Try to infer dimensions from the data shape
        # Assuming square domain if dimensions file is not available
        if field_data.ndim == 2:
            total_cells = field_data.shape[1]
            side_length = int(np.sqrt(total_cells))
            if side_length**2 == total_cells:
                nx, ny = side_length, side_length
            else:
                print(f"Cannot infer dimensions from data shape {field_data.shape}")
                return None
        else:
            print(f"Unexpected data shape: {field_data.shape}")
            return None
    
    print(f"Reshaping data to dimensions: ({ny}, {nx})")
    
    # Reshape the data
    if field_data.ndim == 2:
        # Multiple frames (frames, cells)
        n_frames = field_data.shape[0]
        reshaped_data = np.zeros((n_frames, ny, nx), dtype=field_data.dtype)
        
        for i in range(n_frames):
            try:
                # Reshape and apply vertical flip for correct orientation
                reshaped_data[i] = np.flipud(field_data[i].reshape(ny, nx))
            except Exception as e:
                print(f"Error reshaping frame {i}: {e}")
                return None
    else:
        # Single frame
        try:
            reshaped_data = np.flipud(field_data.reshape(ny, nx))
        except Exception as e:
            print(f"Error reshaping data: {e}")
            return None
    
    return reshaped_data

# Reshape the velocity data
if 'velocity_data' in locals() and velocity_data is not None:
    # Check for dimensions file
    dimensions_file = processed_data_folder / 'dimensions.npz'
    
    # Reshape the data
    reshaped_velocity = reshape_field_data(velocity_data, dimensions_file)
    
    if reshaped_velocity is not None:
        print(f"\nReshaped velocity data:")
        print(f"- Shape: {reshaped_velocity.shape}")
        
        # Convert frame indices to simulation time if we have dt
        if 'key_params' in locals() and key_params:
            # Get the first parameter set
            first_param = next(iter(key_params.values()))
            if 'dt' in first_param:
                dt = first_param['dt']
                # Create time mapping
                frame_to_time = {frame: frame * dt for frame in frame_numbers}
                print(f"\nConverted frames to simulation time using dt = {dt}")
                print(f"Time range: {frame_to_time[frame_numbers[0]]} to {frame_to_time[frame_numbers[-1]]}")
else:
    print("No velocity data available for reshaping")

## Basic Data Visualization

Let's create visualizations of the velocity magnitude data at different time points.